In [3]:
import numpy as np
DTYPE = np.float64
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import pickle
import gzip
from tqdm import tqdm
try:
    import flory
    
except ImportError:
    print("Installing 'flory' temporarily...")
    !pip install flory --quiet
    import flory

from numba import jit

In [42]:
# Global concentrations
phi_global = np.array([0.3, 0.2, 0.2, 0.3], dtype = DTYPE)

# For phi grid
step_size = 0.001
n_points = 100

Xs = np.arange(2, 10, 0.5)
# Xs = [0]
tol = DTYPE(1e-4)

In [43]:
# Flory Huggins Free Energy function
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

In [46]:
# saving dataframes for the 3 phase system

# A new system for each X
for X in (Xs):
    # Interaction matrix
    chis = np.array([[0, 2.0+X, 1.0, -2], [2.0+X, 0.0, -1.0, 1.0], [1.0, -1.0, 0.0, 1.0], [-2, 1, 1, 0]], dtype = DTYPE)
    print(chis)
    
    # Equilibrium 4 phase solution
    phases = flory.find_coexisting_phases(4, chis, phi_global, progress=False)
    
    print("\n" + f"Equlibrium phase volumes: {phases.volumes}")
    print("Equlibrium phase concentrations:\n" + f"{phases.fractions}")
    print()
    # Take the transpose as the notation is different:
    # For me: phi_ij = component i in compartment j
    # For flory: phi_ji = component i in compartment j (OR I THINK SO!)
    
    # vols = phases.volumes
    
    # phases.fractions = np.transpose(phases.fractions)
    # concs = phases.fractions

    # # Compute the system's phase separated Free Energy'
    # # Free energy of the 3 phase system
    
    # F_flory = 0
    # for i in range(len(phases.volumes)):
    #     F_flory += phases.volumes[i]*floryHuggins(phases.fractions[:, i], chis)
    
    # # print(F_flory)
    # output_filepath = f"data/withVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/"
    # output_filename = f"FLORY_3phase_solution.pkl"
    # if not os.path.exists(output_filepath):
    #     os.makedirs(output_filepath)
    # output_file = os.path.join(output_filepath, output_filename)
    
    # with gzip.open(output_file, "wb") as f:
    #     pickle.dump({
    #         "chis": chis,
    #         "phi_global": phi_global,
    #         "phase_fractions": concs,
    #         "phase_volumes": vols,
    #         "flory_free_energy": F_flory,
    #         "_metadata": {"chis-> the interaction matrix",
    #                       "phi_global-> global concentration of components",
    #                       "phase_fractions-> concentration of components in each compartment after running FLory",
    #                       "phase_volumes-> volumes of each compartment after running Flory",
    #                       "flory_free_energy-> free energy of the 3 componenet, 3 compartment solution (outputted from Flory)"
    #                      }
    #     }, f, protocol=pickle.HIGHEST_PROTOCOL)

[[ 0.  4.  1. -2.]
 [ 4.  0. -1.  1.]
 [ 1. -1.  0.  1.]
 [-2.  1.  1.  0.]]

Equlibrium phase volumes: [0.63960937 0.36039063]
Equlibrium phase concentrations:
[[0.44700622 0.04368893 0.10025878 0.40904604]
 [0.03909823 0.47741571 0.37701741 0.10646869]]

[[ 0.   4.5  1.  -2. ]
 [ 4.5  0.  -1.   1. ]
 [ 1.  -1.   0.   1. ]
 [-2.   1.   1.   0. ]]

Equlibrium phase volumes: [0.63290805 0.36709195]
Equlibrium phase concentrations:
[[0.45757257 0.03218233 0.09492178 0.41532336]
 [0.02832706 0.48933666 0.38116675 0.10116948]]

[[ 0.  5.  1. -2.]
 [ 5.  0. -1.  1.]
 [ 1. -1.  0.  1.]
 [-2.  1.  1.  0.]]

Equlibrium phase volumes: [0.62843293 0.37156707]
Equlibrium phase concentrations:
[[0.46509869 0.02397655 0.09126029 0.41966446]
 [0.02076791 0.4977092  0.38391193 0.097611  ]]

[[ 0.   5.5  1.  -2. ]
 [ 5.5  0.  -1.   1. ]
 [ 1.  -1.   0.   1. ]
 [-2.   1.   1.   0. ]]

Equlibrium phase volumes: [0.62531219 0.37468781]
Equlibrium phase concentrations:
[[0.4705516  0.01802749 0.08867611 0